# Review Exercise: InClass Kaggle Competition

In [2]:
# Set environment params
Sys.setenv(LANG='en')  # English

# Import libraries
library(data.table)    # Data manipulate
library(caret)         # ML toolkit
library(mlr)           # ML toolkit
install.packages("party")
library(party)         # Data processing
library(pROC)          # AUC, ROC
library(rpart)         # Decision tree
library(randomForest)  # Bagging and RF
library(gbm)           # Boosting tree

# Resize plot
library(repr)  # String and binary representations
#options(repr.plot.width=4, repr.plot.height=4)

R.version.string

also installing the dependencies 'TH.data', 'libcoin', 'matrixStats', 'multcomp', 'mvtnorm', 'modeltools', 'strucchange', 'coin', 'sandwich'




package 'TH.data' successfully unpacked and MD5 sums checked
package 'libcoin' successfully unpacked and MD5 sums checked
package 'matrixStats' successfully unpacked and MD5 sums checked
package 'multcomp' successfully unpacked and MD5 sums checked
package 'mvtnorm' successfully unpacked and MD5 sums checked
package 'modeltools' successfully unpacked and MD5 sums checked
package 'strucchange' successfully unpacked and MD5 sums checked
package 'coin' successfully unpacked and MD5 sums checked
package 'sandwich' successfully unpacked and MD5 sums checked
package 'party' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\warfaoui\AppData\Local\Temp\RtmpEdwWGh\downloaded_packages


Loading required package: grid

Loading required package: mvtnorm

Loading required package: modeltools

Loading required package: stats4

Loading required package: strucchange

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: sandwich

Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


Loaded gbm 2.1.8



[1] "R version 4.0.5 (2021-03-31)"

### 1. Import and prepare data

#### 1.1. Train data

In [3]:
# Read train data
train <- read.csv('./data/Kaggle/g1/bank_mkt_train.csv')

In [4]:
# Show the data
head(train)

,client_id,age,job,marital,education,default,housing,loan,contact,month,⋯,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,subscribe
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,29925,42,management,married,basic.9y,no,no,no,cellular,jul,⋯,1,999,0,nonexistent,1.4,93.918,-42.7,4.968,5228.1,0
2,37529,35,unemployed,married,university.degree,no,yes,no,telephone,jun,⋯,4,999,0,nonexistent,1.4,94.465,-41.8,4.960,5228.1,0
3,2757,44,technician,married,basic.9y,no,yes,yes,cellular,may,⋯,1,999,0,nonexistent,-1.8,92.893,-46.2,1.264,5099.1,0
4,9642,45,services,married,high.school,no,yes,no,cellular,apr,⋯,1,999,0,nonexistent,-1.8,93.075,-47.1,1.453,5099.1,0
5,14183,45,unknown,married,unknown,unknown,unknown,unknown,telephone,may,⋯,1,999,0,nonexistent,1.1,93.994,-36.4,4.859,5191.0,0
6,15180,38,technician,married,professional.course,no,no,no,telephone,may,⋯,2,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0,0


Separate variables by type.

In [5]:
# List of numerical and catergorical variables
iv_vars <- colnames(train)[2:(ncol(train)-1)]
dv_var <- colnames(train)[ncol(train)]
num_vars <- names(which(unlist(lapply(train[iv_vars], is.numeric)) == TRUE))
cat_vars <- names(which(unlist(lapply(train[iv_vars], is.numeric)) == FALSE))

Check and correct data errors by variable type.

In [6]:
# Check missing values
print(paste("Total NAs:", sum(apply(is.na(train[iv_vars]), 2, sum))))
apply(is.na(train[iv_vars]), 2, sum)

[1] "Total NAs: 3664"


age            job        marital      education        default 
           202            161            199            170            214 
       housing           loan        contact          month    day_of_week 
           195            219            217            199            185 
      campaign          pdays       previous       poutcome   emp.var.rate 
           203            185            209            175            165 
cons.price.idx  cons.conf.idx      euribor3m    nr.employed 
           181            197            204            184

In [8]:
# Filling missing values
for (v in num_vars) { train[is.na(train[v]), v] <- 0 }  # Numeric = 0
for (v in cat_vars) { train[is.na(train[v]), v] <- "Missing" }  # Categorical = 'Missing'

Transform some variables by type.

In [9]:
# Build the dummy encoding on train
encoding <- caret::dummyVars(~ ., data=train[cat_vars], sep="_", fullRank=TRUE)
# Create dummy variables
train_dummy <- as.data.frame(predict(encoding, newdata=train[cat_vars]))
# Correct columns names
names(train_dummy) <- gsub("-", "_", names(train_dummy))
# Combine with numerical variables
train_processed <- cbind(train[num_vars], train_dummy, train[dv_var])

In [10]:
# Check variables names
names(train_processed)

[1] "age"                          "campaign"                    
 [3] "pdays"                        "previous"                    
 [5] "emp.var.rate"                 "cons.price.idx"              
 [7] "cons.conf.idx"                "euribor3m"                   
 [9] "nr.employed"                  "jobblue_collar"              
[11] "jobentrepreneur"              "jobhousemaid"                
[13] "jobmanagement"                "jobMissing"                  
[15] "jobretired"                   "jobself_employed"            
[17] "jobservices"                  "jobstudent"                  
[19] "jobtechnician"                "jobunemployed"               
[21] "jobunknown"                   "maritalmarried"              
[23] "maritalMissing"               "maritalsingle"               
[25] "maritalunknown"               "educationbasic.6y"           
[27] "educationbasic.9y"            "educationhigh.school"        
[29] "educationilliterate"          "educationMissing"            
[31] "educationprofessional.course" "educationuniversity.degree"  
[33] "educationunknown"             "defaultno"                   
[35] "defaultunknown"               "defaultyes"                  
[37] "housingno"                    "housingunknown"              
[39] "housingyes"                   "loanno"                      
[41] "loanunknown"                  "loanyes"                     
[43] "contactMissing"               "contacttelephone"            
[45] "monthaug"                     "monthdec"                    
[47] "monthjul"                     "monthjun"                    
[49] "monthmar"                     "monthmay"                    
[51] "monthMissing"                 "monthnov"                    
[53] "monthoct"                     "monthsep"                    
[55] "day_of_weekMissing"           "day_of_weekmon"              
[57] "day_of_weekthu"               "day_of_weektue"              
[59] "day_of_weekwed"               "poutcomeMissing"             
[61] "poutcomenonexistent"          "poutcomesuccess"             
[63] "subscribe"

#### 1.2. Test data

In [11]:
# Read test data
test <- read.csv('./data/Kaggle/g1/bank_mkt_test.csv')

Check and correct data errors by variable type.

In [12]:
# Check missing values
print(paste("Total NAs:", sum(apply(is.na(test[iv_vars]), 2, sum))))
apply(is.na(test[iv_vars]), 2, sum)

[1] "Total NAs: 1975"


age            job        marital      education        default 
           105            111             95             97            124 
       housing           loan        contact          month    day_of_week 
           103            109             87            120            106 
      campaign          pdays       previous       poutcome   emp.var.rate 
            88            100            105            105            105 
cons.price.idx  cons.conf.idx      euribor3m    nr.employed 
           101            104             88            122

In [13]:
# Filling missing values
for (v in num_vars) { test[is.na(test[v]), v] <- 0 }  # Numeric = 0
for (v in cat_vars) { test[is.na(test[v]), v] <- "Missing" }  # Categorical = 'Missing'

Transform some variables by type.

In [14]:
# Create dummy variables
test_dummy <- as.data.frame(predict(encoding, newdata=test[cat_vars]))
# Correct columns names
names(test_dummy) <- gsub("-", "_", names(test_dummy))
# Combine with numerical variables
test_processed <- cbind(test[num_vars], test_dummy)

#### 1.3. Finalize train, test

Remove independent variables that exist only in train or test.

In [15]:
# List of independent variables to drop
drop_vars <- setdiff(setdiff(names(train_processed), names(test_processed)), dv_var)
print(paste("Drop:", drop_vars))

# Remove from train
train_processed[drop_vars] <- NULL
test_processed[drop_vars] <- NULL

[1] "Drop: defaultyes"


### 2. Train ML model

Reference:
- mlr integrated learners: https://mlr.mlr-org.com/articles/tutorial/integrated_learners.html

#### 2.1. Logistic Regression

In [16]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=20, predict="both")

# Define the model
learner <- makeLearner("classif.logreg", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_mkt", data=train_processed, target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet()
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
                      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

Resampling: cross-validation

Measures:             auc.train   auc.test    

[Resample] iter 1:    0.7869959   0.8281160   

[Resample] iter 2:    0.7892520   0.7777833   

[Resample] iter 3:    0.7879058   0.8101532   

[Resample] iter 4:    0.7888427   0.8066781   

[Resample] iter 5:    0.7912897   0.7513818   

[Resample] iter 6:    0.7896663   0.7838074   

[Resample] iter 7:    0.7897006   0.7780165   

[Resample] iter 8:    0.7897897   0.7851188   

[Resample] iter 9:    0.7914596   0.7418994   

[Resample] iter 10:   0.7882390   0.8135399   

[Resample] iter 11:   0.7881625   0.8079438   

[Resample] iter 12:   0.7913900   0.7602046   

[Resample] iter 13:   0.7913950   0.7549225   

[Resample] iter 14:   0.7893254   0.7808376   

[Resample] iter 15:   0.7911848   0.7485883   

[Resample] iter 16:   0.7874064   0.8229715   

[Resample] iter 17:   0.7883076   0.8141594   

[Resample] iter 18:   0.7913632   0.7556744   

[Resample] iter 19:   0.7911161   0.7586077   

[Resample]

In [17]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)
best_md

Model for learner.id=classif.logreg; learner.class=classif.logreg
Trained on: task.id = bank_mkt; obs = 20000; features = 61
Hyperparameters: model=FALSE

In [18]:
# Summary the best model
summary(best_md$learner.model)


Call:
stats::glm(formula = f, family = "binomial", data = getTaskData(.task, 
    .subset), weights = .weights, model = FALSE)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1985  -0.4219  -0.3330  -0.2384   3.0692  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -4.305e-01  6.868e-01  -0.627 0.530806    
age                           4.681e-03  2.717e-03   1.723 0.084934 .  
campaign                     -4.620e-02  1.277e-02  -3.619 0.000296 ***
pdays                        -5.783e-04  1.859e-04  -3.110 0.001871 ** 
previous                      2.410e-01  7.407e-02   3.254 0.001139 ** 
emp.var.rate                 -1.424e-01  3.949e-02  -3.605 0.000312 ***
cons.price.idx               -3.232e-03  2.739e-03  -1.180 0.238008    
cons.conf.idx                 9.303e-03  4.213e-03   2.208 0.027218 *  
euribor3m                    -3.248e-01  3.658e-02  -8.880  < 2e-16 ***
nr.employed            

In [19]:
# Make prediction on test (holdout) data
pred <- predict(best_md, newdata=test_processed)
pred

Prediction: 10000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 0.02
     prob.0     prob.1 response
1 0.9073187 0.09268130        0
2 0.9438906 0.05610942        0
3 0.6093050 0.39069499        0
4 0.8924933 0.10750669        0
5 0.8828813 0.11711869        0
6 0.3790033 0.62099669        1
... (#rows: 10000, #cols: 3)

In [20]:
# Output predicted file
output <- cbind(test[c(1)], subscribe=pred$data$prob.1)
write.csv(output, './data/Kaggle/lr_submission_1.csv', row.names=FALSE)

#### 2.2. Decision Tree

In [21]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=20, predict="both")

# Define the model
learner <- makeLearner("classif.rpart", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_mkt", data=train_processed, target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
    makeDiscreteParam('maxdepth', value=c(2, 5, 10)),
    makeDiscreteParam('minsplit', value=c(10, 20))
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
                      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

[Tune] Started tuning learner classif.rpart for parameter set:

             Type len Def Constr Req Tunable Trafo
maxdepth discrete   -   - 2,5,10   -    TRUE     -
minsplit discrete   -   -  10,20   -    TRUE     -

With control class: TuneControlGrid

Imputation value: -0

[Tune-x] 1: maxdepth=2; minsplit=10

[Tune-y] 1: auc.test.mean=0.7035158; time: 0.1 min

[Tune-x] 2: maxdepth=5; minsplit=10

[Tune-y] 2: auc.test.mean=0.7035158; time: 0.2 min

[Tune-x] 3: maxdepth=10; minsplit=10

[Tune-y] 3: auc.test.mean=0.7035158; time: 0.2 min

[Tune-x] 4: maxdepth=2; minsplit=20

[Tune-y] 4: auc.test.mean=0.7035158; time: 0.1 min

[Tune-x] 5: maxdepth=5; minsplit=20

[Tune-y] 5: auc.test.mean=0.7035158; time: 0.2 min

[Tune-x] 6: maxdepth=10; minsplit=20

[Tune-y] 6: auc.test.mean=0.7035158; time: 0.2 min

[Tune] Result: maxdepth=5; minsplit=20 : auc.test.mean=0.7035158



In [22]:
# Retrain the model with tbe best hyper-parameters
best_md <- mlr::train(best_learner, train_task)
best_md

Model for learner.id=classif.rpart; learner.class=classif.rpart
Trained on: task.id = bank_mkt; obs = 20000; features = 61
Hyperparameters: xval=0

In [23]:
# Summary the best model
summary(best_md$learner.model)

Call:
rpart::rpart(formula = f, data = d, xval = 0)
  n= 20000 

          CP nsplit rel error
1 0.05702334      0 1.0000000
2 0.01000000      2 0.8859533

Variable importance
    nr.employed       euribor3m    emp.var.rate   cons.conf.idx poutcomesuccess 
             27              20              15              12              11 
 cons.price.idx           pdays 
             10               6 

Node number 1: 20000 observations,    complexity param=0.05702334
  predicted class=0  expected loss=0.11355  P(node) =1
    class counts: 17729  2271
   probabilities: 0.886 0.114 
  left son=2 (17404 obs) right son=3 (2596 obs)
  Primary splits:
      nr.employed     < 5087.65 to the right, improve=559.8677, (0 missing)
      euribor3m       < 1.2395  to the right, improve=512.3372, (0 missing)
      poutcomesuccess < 0.5     to the left,  improve=423.3535, (0 missing)
      pdays           < 513     to the right, improve=356.9348, (0 missing)
      emp.var.rate    < -0.65   to the righ

In [24]:
# Make prediction on test (holdout) data
pred <- predict(best_md, newdata=test_processed)
pred

Prediction: 10000 observations
predict.type: prob
threshold: 0=0.50,1=0.50
time: 0.02
     prob.0     prob.1 response
1 0.9321420 0.06785796        0
2 0.9321420 0.06785796        0
3 0.6661743 0.33382570        0
4 0.9321420 0.06785796        0
5 0.9321420 0.06785796        0
6 0.2707965 0.72920354        1
... (#rows: 10000, #cols: 3)

In [25]:
# Output predicted file
output <- cbind(test[c(1)], subscribe=pred$data$prob.1)
write.csv(output, './data/Kaggle/dt_submission_1.csv', row.names=FALSE)